In [4]:
import numpy as np
import pandas as pd
from pyreadr import read_r

### Firearm Involved Crimes: Robbery, Agg Assault, Murder

In [6]:
keep_crimes = ['robbery', 'aggravated assault', 'murder/nonnegligent manslaughter'] 
dfs = []
for year in range(2010, 2021, 1):
    df = read_r(rf'..\data\raw\nibrs_offense_segment_{str(year)}.rds')[None]
    # filter to VA
    df = df[(df['ucr_offense_code'].isin(keep_crimes))].loc[:]
    dfs.append(df)
offenses = pd.concat(dfs)

In [7]:
offenses.to_csv('../data/processed/nibrs_offense_segments.csv')

In [15]:
def weapon_involved(values):
    '''Accepts a list and checks if any values indicate firearm or gun'''
    check_string = ', '.join([str(value) for value in values]).lower() # ensure it's a string
    if 'firearm' in check_string or 'gun' in check_string or'rifle' in check_string:
        return 1
    else:
        return 0

In [16]:
offenses['firearm_involved'] = offenses.apply(lambda x: weapon_involved([x['type_weapon_force_involved_1'], x['type_weapon_force_involved_2'], x['type_weapon_force_involved_3']]), axis=1)

In [17]:
offenses['firearm_involved'].fillna(0, inplace=True)

In [19]:
offenses.to_csv('..\data\processed\offenses_firearm_indicator.csv')

In [20]:
df = read_r(rf'..\data\raw\nibrs_offense_segment_{str(year)}.rds')[None]

In [22]:
df2 = read_r(rf'..\data\raw\nibrs_property_segment_{str(year)}.rds')[None]

In [23]:
keep_crimes = ['theft from motor vehicle'] 
dfs = []
for year in range(2010, 2021, 1):
    df = read_r(rf'..\data\raw\nibrs_offense_segment_{str(year)}.rds')[None]
    # filter to VA
    df = df[(df['ucr_offense_code'].isin(keep_crimes))].loc[:]
    dfs.append(df)
mvtheft = pd.concat(dfs)

In [25]:
mvtheft.to_csv(rf'..\data\processed\tfmv.csv')

In [39]:
keep_property = ['firearms'] 
keep_incidents = pd.unique(mvtheft['incident_number'])
dfs = []
for year in range(2010, 2021, 1):
    df = read_r(rf'..\data\raw\nibrs_property_segment_{str(year)}.rds')[None]
    # filter to VA
    df = df[(df['property_description'].isin(keep_property)) & (df['type_of_property_loss'].str.lower().str.contains('stolen')) & (df['incident_number'].isin(keep_incidents))].loc[:]
    print(df.shape[0])
    dfs.append(df)
mvtheftfirearms = pd.concat(dfs)

17664
18643
21360
22597
22819
26964
33177
36898
37572
46904
72588


In [41]:
mvtheftfirearms.to_csv('stolen_firearms_raw_prop.csv')

In [47]:
firearms_stolen_by_incident_num = mvtheftfirearms.groupby('incident_number', as_index=False).agg(NumFirearmsStolen=('property_description','count'))

In [46]:
mvtheft.shape[0]

4830353

In [49]:
tfmv_guns = mvtheft.merge(firearms_stolen_by_incident_num, on='incident_number')

In [50]:
tfmv_guns.shape[0]

453485

In [53]:
tfmv_guns.to_csv(rf'..\data\processed\theft_mv_firearms.csv')

### Aggregrating by state, year, month, counts of stolen firearms, and the proportion of firearm crimes

In [2]:
import pandas as pd

In [24]:
offenses = pd.read_csv('..\data\processed\offenses_firearm_indicator.csv', low_memory=False)
tfmv = pd.read_csv(rf'..\data\processed\theft_mv_firearms.csv', low_memory=False)

In [26]:
offenses.columns

Index(['Unnamed: 0', 'ori', 'year', 'state', 'state_abb', 'incident_number',
       'incident_date', 'ucr_offense_code', 'offense_attempted_or_completed',
       'offender_suspected_of_using_1', 'offender_suspected_of_using_2',
       'offender_suspected_of_using_3', 'location_type',
       'number_of_premises_entered', 'method_of_entry',
       'type_criminal_activity_1', 'type_criminal_activity_2',
       'type_criminal_activity_3', 'type_weapon_force_involved_1',
       'automatic_weapon_indicator_1', 'type_weapon_force_involved_2',
       'automatic_weapon_indicator_2', 'type_weapon_force_involved_3',
       'automatic_weapon_indicator_3', 'bias_motivation', 'unique_incident_id',
       'firearm_involved'],
      dtype='object')

In [27]:
offenses['incident_date'] = pd.to_datetime(offenses['incident_date'])
offenses['year'] = offenses['incident_date'].dt.year
offenses['month'] = offenses['incident_date'].dt.month

In [28]:
tfmv.columns

Index(['Unnamed: 0', 'ori', 'year', 'state', 'state_abb', 'incident_number',
       'incident_date', 'ucr_offense_code', 'offense_attempted_or_completed',
       'offender_suspected_of_using_1', 'offender_suspected_of_using_2',
       'offender_suspected_of_using_3', 'location_type',
       'number_of_premises_entered', 'method_of_entry',
       'type_criminal_activity_1', 'type_criminal_activity_2',
       'type_criminal_activity_3', 'type_weapon_force_involved_1',
       'automatic_weapon_indicator_1', 'type_weapon_force_involved_2',
       'automatic_weapon_indicator_2', 'type_weapon_force_involved_3',
       'automatic_weapon_indicator_3', 'bias_motivation', 'unique_incident_id',
       'NumFirearmsStolen'],
      dtype='object')

In [30]:
tfmv['incident_date'] = pd.to_datetime(tfmv['incident_date'])
tfmv['year'] = tfmv['incident_date'].dt.year
tfmv['month'] = tfmv['incident_date'].dt.month

In [31]:
offenses = offenses.groupby(['state', 'year', 'month'], as_index=False).agg(num_firearms=('firearm_involved', 'sum'), total_crimes=('firearm_involved', 'count'))

In [32]:
offenses['perc_firearm_involv'] = offenses['num_firearms']/offenses['total_crimes']

In [33]:
offenses.drop(columns=['num_firearms', 'total_crimes'], inplace=True)

In [34]:
offenses.head()

,state,year,month,perc_firearm_involv
0,alabama,2009,9,0.000000
1,alabama,2009,12,0.000000
2,alabama,2010,1,0.250000
3,alabama,2010,2,0.200000
4,alabama,2010,3,0.571429


In [35]:
tfmv = tfmv.groupby(['state','year', 'month'],as_index=False).agg(stolen_firearms=('NumFirearmsStolen', 'sum'))

In [36]:
tfmv

,state,year,month,stolen_firearms
0,alabama,2010,1,4
1,alabama,2010,2,3
2,alabama,2010,3,2
3,alabama,2010,4,4
4,alabama,2010,5,5
...,...,...,...,...
5051,wyoming,2020,8,5
5052,wyoming,2020,9,2
5053,wyoming,2020,10,11
5054,wyoming,2020,11,5


In [37]:
df = tfmv.merge(offenses, on=['state', 'year', 'month'])

In [38]:
df.to_csv(rf'..\data\processed\stolen_firearms_perc_firearm_involv.csv')

### Add in gun law information

In [92]:
df = pd.read_csv(rf'..\data\processed\stolen_firearms_perc_firearm_involv.csv')

In [94]:
df['YearMonth'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str), format='%Y-%m')

In [95]:
laws = pd.read_csv(rf'..\data\raw\firearm_db.csv')

In [96]:
laws = laws[(laws['Effect']=='Restrictive') & (laws['Type of Change'] == 'Implement') & (laws['Law Class']=='background checks') & (laws['Law Class Subtype'] == 'private sales')]

In [97]:
laws['State'] = laws['State'].str.lower()
laws = laws[['State', 'Effective Date Month', 'Effective Date Year', 'Law ID']]
laws.rename(columns={'Effective Date Year': 'year', 'Effective Date Month': 'month', 'State': 'state'}, inplace=True)
laws['year'] = laws['year'].astype(int)
laws['month'] = laws['month'].astype(int)

In [98]:
laws['YearMonth'] = pd.to_datetime(laws['year'].astype(str) + '-' + laws['month'].astype(str), format='%Y-%m')
laws = laws.loc[laws.groupby(['state'])['YearMonth'].idxmax()]

In [99]:
df = df.merge(laws, how='left', on =['state'])

In [101]:
df.loc[df['Law ID'].isnull(), 'priv_sale_law'] = 0 # if the law doesn't exist for that state hit 0
df.loc[df['Law ID'].notnull(), 'priv_sale_law'] = 1 # else the law does exist for that state so 1

In [102]:
df

,Unnamed: 0,state,year_x,month_x,stolen_firearms,perc_firearm_involv,YearMonth_x,month_y,year_y,Law ID,YearMonth_y,priv_sale_law
0,0,alabama,2010,1,4,0.250000,2010-01-01,NaN,NaN,NaN,NaT,0.0
1,1,alabama,2010,2,3,0.200000,2010-02-01,NaN,NaN,NaN,NaT,0.0
2,2,alabama,2010,3,2,0.571429,2010-03-01,NaN,NaN,NaN,NaT,0.0
3,3,alabama,2010,4,4,0.333333,2010-04-01,NaN,NaN,NaN,NaT,0.0
4,4,alabama,2010,5,5,0.500000,2010-05-01,NaN,NaN,NaN,NaT,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5043,5043,wyoming,2020,8,5,0.166667,2020-08-01,NaN,NaN,NaN,NaT,0.0
5044,5044,wyoming,2020,9,2,0.333333,2020-09-01,NaN,NaN,NaN,NaT,0.0
5045,5045,wyoming,2020,10,11,0.100000,2020-10-01,NaN,NaN,NaN,NaT,0.0
5046,5046,wyoming,2020,11,5,0.176471,2020-11-01,NaN,NaN,NaN,NaT,0.0


In [104]:
df.loc[(df['YearMonth_x'] <= df['YearMonth_y']), 'priv_sale_law'] = 0 # however what if the law was AFTER the crime? mark 0

In [106]:
df.drop(columns=['month_y', 'year_y','YearMonth_y', 'YearMonth_x'], inplace=True)

In [107]:
df.rename(columns={'year_x': 'year', 'month_x': 'month'}, inplace=True)

In [108]:
df['Law ID'].fillna('', inplace=True)

In [111]:
df.to_csv('data.csv')

In [313]:
df = pd.read_csv('data.csv')

In [314]:
df.drop(columns=['Law ID', 'Unnamed: 0', 'priv_sale_law', 'Unnamed: 0.1'], inplace=True)

### Add in Assault Weapon & Domestic

In [315]:
from io import StringIO

In [316]:
assaults = """State	Year	Month	Date
California	1990	1	1
Connecticut	1993	10	1
District of Columbia	1976	9	24
Hawaii	1992	7	1
Maryland	2013	10	1
Massachusetts	1998	10	21
New Jersey	1990	5	30
New York	2000	11	1""" 
assaults = pd.read_csv(StringIO(assaults),sep='\t')

In [317]:
dv = """State	Year	Month	Date
Alaska	1996	7	1
Arizona	1996	7	19
California	2000	1	1
Connecticut	1999	7	1
Hawaii	1996	7	1
Illinois	1994	7	1
Indiana	2002	7	1
Maryland	1996	10	1
Montana	1995	10	1
Nebraska	2004	7	15
New Hampshire	2000	1	1
New Jersey	2004	1	14
Ohio	1994	12	9
Oklahoma	1993	9	1
Pennsylvania	1986	4	16
Tennessee	1995	7	1
Utah	1995	7	1
Vermont	2018	9	1
Washington	2019	7	28
West Virginia	2001	9	1"""
dv = pd.read_csv(StringIO(dv), sep='\t')

In [318]:
private = """state	Year	Month	Date
California	1991	1	1
Colorado	2013	7	1
Connecticut	2000	12	1
Delaware	2013	7	1
Hawaii	1927	7	1
Illinois	1968	7	1
Iowa	1990	4	5
Maryland	1996	10	1
Massachusetts	1969	1	1
Michigan	1927	9	5
Nebraska	1991	9	5
Nevada	2020	1	2
New Jersey	1970	1	1
New Mexico	2019	7	1
New York	2013	3	16
North Carolina	1995	12	1
Oregon	2015	5	11
Pennsylvania	1998	7	1
Rhode Island	1990	7	1
Vermont	2018	4	11
Virginia	2020	7	1
Washington	2014	12	4"""
private = pd.read_csv(StringIO(private), sep='\t')

In [319]:
assaults['assYearMonth'] = pd.to_datetime(assaults['Year'].astype(str) + '-' + assaults['Month'].astype(str), format='%Y-%m')
dv['dvYearMonth'] = pd.to_datetime(dv['Year'].astype(str) + '-' + dv['Month'].astype(str), format='%Y-%m')
assaults['state'] = assaults['State'].str.lower()
dv['state'] = dv['State'].str.lower()
private['privYearMonth'] = pd.to_datetime(private['Year'].astype(str) + '-' + private['Month'].astype(str), format='%Y-%m')
private['state'] = private['state'].str.lower()

In [320]:
assaults.drop(columns=['State'], inplace=True)
dv.drop(columns=['State'], inplace=True)

In [321]:
df = df.merge(assaults, on='state', how='left')
df = df.merge(dv, on='state', how='left')
df = df.merge(private, on='state', how='left')

In [323]:
df.drop(columns=['Year_x', 'Month_x', 'Year_y', 'Date_y', 'Date_x', 'Month_y', 'Year', 'Month', "Date"], inplace=True)

In [324]:
df['YearMonth'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str), format='%Y-%m')

In [362]:
df.loc[df['assYearMonth'].isna(), 'assault_law'] = 0 # if the law doesn't exist for that state hit 0
df.loc[df['assYearMonth'].notna(), 'assault_law'] = 1 # else the law does exist for that state so 1
df.loc[df['assYearMonth'] >= df['YearMonth'], 'assault_law'] = 0 # however what if the law was AFTER the crime? mark 0

In [363]:
pd.unique(df['assault_law'])

array([0., 1.])

In [339]:
df.loc[df['dvYearMonth'].isna(), 'dv_law'] = 0 # if the law doesn't exist for that state hit 0
df.loc[df['dvYearMonth'].notna(), 'dv_law'] = 1 # else the law does exist for that state so 1
df.loc[df['dvYearMonth'] >= df['YearMonth'], 'dv_law'] = 0 # however what if the law was AFTER the crime? mark 0

In [340]:
df.loc[df['privYearMonth'].isna(), 'priv_law'] = 0 # if the law doesn't exist for that state hit 0
df.loc[df['privYearMonth'].notna(), 'priv_law'] = 1 # else the law does exist for that state so 1
df.loc[df['privYearMonth'] >= df['YearMonth'], 'priv_law'] = 0 # however what if the law was AFTER the crime? mark 0

In [365]:
df[(df['assault_law']==1)].groupby(['state', 'year']).agg(privSum=('assault_law', 'sum')).head(50)

privSum
state                year         
connecticut          2009      2.0
                     2010     12.0
                     2011     12.0
                     2012     12.0
                     2013     12.0
                     2014     12.0
                     2015     12.0
                     2016     12.0
                     2017     12.0
                     2018     12.0
                     2019     12.0
                     2020     12.0
district of columbia 2010      1.0
                     2011      1.0
                     2016      2.0
                     2017      6.0
                     2018      1.0
                     2019      2.0
hawaii               2018      8.0
                     2019      6.0
                     2020      7.0
maryland             2016      2.0
                     2017     12.0
                     2018     12.0
                     2019     12.0
                     2020     12.0
massachusetts        2009      1.0
                     2010     12.0
                     2011     12.0
                     2012     12.0
                     2013     12.0
                     2014     12.0
                     2015     12.0
                     2016     12.0
                     2017     12.0
                     2018     12.0
                     2019     12.0
                     2020     12.0
new york             2020     12.0

In [371]:
df[df['state']=='connecticut']

,state,year,month,stolen_firearms,perc_firearm_involv,assYearMonth,dvYearMonth,privYearMonth,YearMonth,assault_law,dv_law,priv_law
530,connecticut,2009,6,1,0.142857,1993-10-01,1999-07-01,2000-12-01,2009-06-01,1.0,1.0,1.0
531,connecticut,2009,9,1,0.227273,1993-10-01,1999-07-01,2000-12-01,2009-09-01,1.0,1.0,1.0
532,connecticut,2010,1,60,0.227642,1993-10-01,1999-07-01,2000-12-01,2010-01-01,1.0,1.0,1.0
533,connecticut,2010,2,38,0.228782,1993-10-01,1999-07-01,2000-12-01,2010-02-01,1.0,1.0,1.0
534,connecticut,2010,3,20,0.216438,1993-10-01,1999-07-01,2000-12-01,2010-03-01,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
659,connecticut,2020,8,64,0.336449,1993-10-01,1999-07-01,2000-12-01,2020-08-01,1.0,1.0,1.0
660,connecticut,2020,9,57,0.296954,1993-10-01,1999-07-01,2000-12-01,2020-09-01,1.0,1.0,1.0
661,connecticut,2020,10,39,0.337156,1993-10-01,1999-07-01,2000-12-01,2020-10-01,1.0,1.0,1.0
662,connecticut,2020,11,33,0.334405,1993-10-01,1999-07-01,2000-12-01,2020-11-01,1.0,1.0,1.0


In [372]:
df.drop(columns=['assYearMonth', 'dvYearMonth', 'YearMonth', 'privYearMonth'], inplace=True)


In [373]:
df

,state,year,month,stolen_firearms,perc_firearm_involv,assault_law,dv_law,priv_law
0,alabama,2010,1,4,0.250000,0.0,0.0,0.0
1,alabama,2010,2,3,0.200000,0.0,0.0,0.0
2,alabama,2010,3,2,0.571429,0.0,0.0,0.0
3,alabama,2010,4,4,0.333333,0.0,0.0,0.0
4,alabama,2010,5,5,0.500000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
5043,wyoming,2020,8,5,0.166667,0.0,0.0,0.0
5044,wyoming,2020,9,2,0.333333,0.0,0.0,0.0
5045,wyoming,2020,10,11,0.100000,0.0,0.0,0.0
5046,wyoming,2020,11,5,0.176471,0.0,0.0,0.0


In [374]:
df.to_csv('final_data.csv')